<a href="https://colab.research.google.com/github/seawavve/CNN_wavve/blob/main/onnx_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train MNIST

In [1]:
'''
Test loss: 0.023477373644709587
Test accuracy: 0.9954000115394592
'''
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
from keras.utils import np_utils
import keras
import tensorflow.keras as tk
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np 
from keras.callbacks import EarlyStopping, ModelCheckpoint

print('Python version : ', sys.version)
print('Keras version : ', keras.__version__)

img_rows = 28
img_cols = 28
(x_train, y_train), (x_test, y_test) = tk.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 10
filename='checkpoint'.format(epochs,batch_size)
early_stopping=EarlyStopping(monitor='val_loss',mode='min',patience=15,verbose=1)                           #얼리스타핑
checkpoint=ModelCheckpoint(filename,monitor='val_loss',verbose=1,save_best_only=True,mode='auto')           #체크포인트

y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

model = Sequential()
#1
model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(0.25))
#2
model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
#3
model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

#4
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test) , callbacks=[checkpoint,early_stopping])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:',  score[0])
print('Test accuracy:', score[1])
# model.save('MNIST_CNN_model')

Python version :  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
Keras version :  2.7.0
11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 28, 28, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 batch_normalization_1

# Load Model

In [2]:
from keras.models import load_model
model = load_model('./checkpoint')

from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_shapes.png', show_shapes=True)
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy=", score[1])

Accuracy= 0.9937000274658203


# Transfer TF to ONNX

In [3]:
! pip install -U tf2onnx
! python -m tf2onnx.convert --saved-model checkpoint --output model.onnx

     |████████████████████████████████| 435 kB 5.0 MB/s 
     |████████████████████████████████| 12.7 MB 63.3 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0


/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-01-18 11:59:15,983 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-01-18 11:59:16,658 - INFO - Signatures found in model: [serving_default].
2022-01-18 11:59:16,658 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-01-18 11:59:16,658 - INFO - Output names: ['dense_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-01-18 11:59:16,937 - WARNING - From /usr/local/lib/python3.7/dist-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-01-18 11:59:17,104 - INFO - Using ten

# ONNX to TensorRT

In [7]:
! python --version

Python 3.7.12


In [9]:
! python3 -m pip install onnx==1.8.0

     |████████████████████████████████| 7.7 MB 5.1 MB/s 
  Attempting uninstall: onnx
    Found existing installation: onnx 1.10.2
    Uninstalling onnx-1.10.2:
      Successfully uninstalled onnx-1.10.2


In [12]:
# import onnx
# import onnx_tensorrt.backend as backend
# import numpy as np

# model = onnx.load("model.onnx")
# engine = backend.prepare(model, device='CUDA:0')
# input_data = np.random.random(size=(32, 3, 224, 224)).astype(np.float32)
# output_data = engine.run(input_data)[0]
# print(output_data)
# print(output_data.shape)